# Generate single-cell crops for min/max montages of the top feature (Zernike phase 2,2) for both stains

## Import libraries

In [1]:
import pathlib
from pprint import pprint
from typing import List

import cv2
import pandas as pd

## Set functions for processing and generating single-cell crops

In [2]:
# Function for formatting min/max row data frames into dictionaries
def create_sc_dict(dfs: List[pd.DataFrame], names: List[str]) -> dict:
    """Format lists of data frames and names into a dictionary with all relevant metadata to find single-cell images.

    Args:
        dfs (List[pd.DataFrame]): List of data frames each containing a single cell and relevant metadata.
        names (List[str]): List of names corresponding to the data frames.

    Returns:
        dict: Dictionary containing info relevant for finding single-cell crops.
    """
    sc_dict = {}
    for df, name in zip(dfs, names):
        for i, (_, row) in enumerate(df.iterrows()):
            if i >= 5:  # Limit to 5 rows
                break
            key = f"{name}_{i+1}"  # Create key with incrementing number
            sc_dict[key] = {
                "plate": row["Metadata_Plate"],
                "well": row["Metadata_Well"],
                "site": row["Metadata_Site"],
                "location_center_x": row["Metadata_Nuclei_Location_Center_X"],
                "location_center_y": row["Metadata_Nuclei_Location_Center_Y"],
            }
    return sc_dict

In [3]:
# Function for generating and saving single-cell crops per channel as PNGs
def generate_sc_crops(
    sc_dict: dict,
    images_dir: pathlib.Path,
    output_img_dir: pathlib.Path,
    crop_size: int,
) -> None:
    """Using a dictionary with single-cell metadata info per image set, single-cell crops per channel are generated 
    and saved as PNGs in an image set folder.

    Args:
        sc_dict (dict): Dictionary containing info relevant for finding single-cell crops.
        images_dir (pathlib.Path): Directory where illumination corrected images are found.
        output_img_dir (pathlib.Path): Main directory to save each image set single-cell crops
        crop_size (int): Size of the box in pixels (example: setting crop_size as 250 will make a 250x250 pixel crop around the single-cell center coordinates)
    """
    for key, info in sc_dict.items():
        # Initialize a list to store file paths for every image set
        file_paths = []

        # Create file paths with well, site, and channel
        for i in range(3):  # Update the range to start from 0 and end at 2
            filename = f"{images_dir}/{info['plate']}/{info['plate']}_{info['well']}_{info['site']}_CH{i}_Z09_illumcorrect.tiff"
            file_paths.append(filename)

            # Read the image
            channel_image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

            # Use the location_center_x and location_center_y to create a crop
            center_x = info.get("location_center_x")
            center_y = info.get("location_center_y")

            # Crop dimensions (including crop_size)
            half_crop = crop_size // 2

            # Ensure the center coordinates are valid
            if center_x is not None and center_y is not None:
                # Calculate crop boundaries
                top_left_x = max(int(center_x - half_crop), 0)
                top_left_y = max(int(center_y - half_crop), 0)
                bottom_right_x = min(int(center_x + half_crop), channel_image.shape[1])
                bottom_right_y = min(int(center_y + half_crop), channel_image.shape[0])

                # Perform cropping
                cropped_channel = channel_image[
                    top_left_y:bottom_right_y, top_left_x:bottom_right_x
                ]

                # Ensure the cropped image is of size 250x250
                cropped_channel = cv2.resize(cropped_channel, (crop_size, crop_size))

                # Make directory for the key to keep all channels for an image in one folder
                key_dir = pathlib.Path(f"{output_img_dir}/{key}")
                key_dir.mkdir(exist_ok=True, parents=True)

                # Save the cropped image with single_cell and channel information
                output_filename = str(pathlib.Path(f"{key_dir}/{key}_CH{i}_cropped.png"))
                cv2.imwrite(output_filename, cropped_channel)

## Set paths and variables

In [4]:
# Images directory for 
images_dir = pathlib.Path(
    "/media/18tbdrive/Github_Repositories/nuclear_speckle_image_profiling/2.illumination_correction/IC_corrected_images"
).resolve(strict=True)

# Output dir for cropped images
output_img_dir = pathlib.Path("./sc_crops")
output_img_dir.mkdir(exist_ok=True)

# Define the size of the cropping box (250x250 pixels)
crop_size = 250

# Create open list for one row data frames for each top feature per channel per cell type
list_of_dfs = []

# Create open list of names to assign each data frame in a list relating to the feature, channel, and cell type
list_of_names = []

## Load in training data to use to find example crops

In [5]:
model_dir = pathlib.Path('./models')  # Path to the folder with models

# paths to the data splits
training_data_path = pathlib.Path('./data/training_data.parquet')

# Load your datasets
training_df = pd.read_parquet(training_data_path)

print(training_df.shape)
training_df.head()

(43406, 574)


,Metadata_CellLine,Metadata_Condition,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Nuclei_Site_Count,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Nuclei_AreaShape_Area,...,Nuclei_Texture_Variance_A647_3_02_256,Nuclei_Texture_Variance_A647_3_03_256,Nuclei_Texture_Variance_DAPI_3_00_256,Nuclei_Texture_Variance_DAPI_3_01_256,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GOLD_3_00_256,Nuclei_Texture_Variance_GOLD_3_01_256,Nuclei_Texture_Variance_GOLD_3_02_256,Nuclei_Texture_Variance_GOLD_3_03_256
0,786O,TMEM259 kd5,214,slide2,B3,M35,119,194.287533,47.829652,1.989316,...,0.053548,0.058190,0.438567,0.417687,0.440925,0.436351,-0.102935,-0.106342,-0.103687,-0.101349
1,786O,FIBP kd7,125,slide1,A4,M36,106,1739.206684,1157.896218,-0.684056,...,0.895503,0.910324,2.924820,2.886479,2.890870,2.927758,0.519375,0.512958,0.509951,0.521149
2,786O,SART1 kd4,226,slide3,B2,M15,70,437.178872,1632.926118,-0.254211,...,2.176894,2.019121,-0.022723,-0.017170,-0.017412,-0.010453,1.830279,1.929267,1.950886,1.836266
3,786O,FIBP kd6,282,slide3,B3,M13,223,1916.067522,2103.329507,-0.369706,...,0.211106,0.213585,0.065390,0.055730,0.046092,0.045562,0.066281,0.070989,0.067091,0.065943
4,786O,SART1 kd4,205,slide1,B2,M37,31,1478.593750,2230.404891,-0.003261,...,0.547837,0.534642,0.074996,0.082636,0.070279,0.087246,0.303568,0.299110,0.285290,0.287522


## Find min/max representative single-cells per stain

In [6]:
# Get data frame with the top single-cells
top_A647 = training_df.nlargest(5, "Nuclei_RadialDistribution_ZernikePhase_A647_2_2")[
    [
        "Nuclei_RadialDistribution_ZernikePhase_A647_2_2",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_Condition",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_A647)
list_of_names.append("top_A647")

print(top_A647.shape)
top_A647

(5, 7)


,Nuclei_RadialDistribution_ZernikePhase_A647_2_2,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Condition
32456,1.741682,A2,slide3,M34,1571.021560,1079.886239,ALY kd8
10389,1.741180,B2,slide3,M26,209.552416,1256.926766,SART1 kd4
29975,1.741099,A2,slide3,M59,971.928319,109.738938,ALY kd8
2729,1.740969,B3,slide3,M12,1690.118067,1312.709192,FIBP kd6
8012,1.740699,B3,slide3,M5,436.688252,1402.459759,FIBP kd6


In [7]:
# Get data frame with the bottom single-cells
bottom_A647 = training_df.nsmallest(5, "Nuclei_RadialDistribution_ZernikePhase_A647_2_2")[
    [
        "Nuclei_RadialDistribution_ZernikePhase_A647_2_2",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_Condition",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(bottom_A647)
list_of_names.append("bottom_A647")

print(bottom_A647.shape)
bottom_A647

(5, 7)


,Nuclei_RadialDistribution_ZernikePhase_A647_2_2,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Condition
13359,-1.740841,B2,slide3,M56,549.406841,691.427543,SART1 kd4
16814,-1.740802,B2,slide3,M17,1343.588618,290.339024,SART1 kd4
20041,-1.740741,A2,slide3,M42,1506.635486,649.154496,ALY kd8
34888,-1.739866,A2,slide3,M29,194.426029,2079.284189,ALY kd8
30644,-1.739400,A2,slide3,M11,1160.286360,225.751581,ALY kd8


In [8]:
# Get data frame with the top single-cells
top_GOLD = training_df.nlargest(5, "Nuclei_RadialDistribution_ZernikePhase_GOLD_2_2")[
    [
        "Nuclei_RadialDistribution_ZernikePhase_GOLD_2_2",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_Condition",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_GOLD)
list_of_names.append("top_GOLD")

print(top_GOLD.shape)
top_GOLD

(5, 7)


,Nuclei_RadialDistribution_ZernikePhase_GOLD_2_2,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Condition
25821,1.743746,A2,slide3,M42,452.423150,1892.726123,ALY kd8
2729,1.743507,B3,slide3,M12,1690.118067,1312.709192,FIBP kd6
3176,1.743133,B3,slide3,M27,772.062628,1620.917631,FIBP kd6
17525,1.742628,B2,slide3,M58,1216.519623,1319.047881,SART1 kd4
8785,1.742544,B3,slide3,M43,151.123913,1391.723913,FIBP kd6


In [9]:
# Get data frame with the bottom single-cells
bottom_GOLD = training_df.nsmallest(5, "Nuclei_RadialDistribution_ZernikePhase_GOLD_2_2")[
    [
        "Nuclei_RadialDistribution_ZernikePhase_GOLD_2_2",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_Condition",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(bottom_GOLD)
list_of_names.append("bottom_GOLD")

print(bottom_GOLD.shape)
bottom_GOLD

(5, 7)


,Nuclei_RadialDistribution_ZernikePhase_GOLD_2_2,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Condition
10033,-1.741537,A3,slide3,M15,1860.994183,1613.974404,SART1 kd6
3048,-1.741368,B3,slide3,M39,2193.645659,407.364357,FIBP kd6
9682,-1.741223,B2,slide3,M60,827.349682,1569.164396,SART1 kd4
6586,-1.741222,A3,slide3,M5,2187.984337,1964.663855,SART1 kd6
22532,-1.741221,B3,slide3,M5,424.517241,1283.706034,FIBP kd6


## Generate dictionary with all representative single-cells

In [10]:
sc_dict = create_sc_dict(dfs=list_of_dfs, names=list_of_names)

# Check the created dictionary for the first two items
pprint(list(sc_dict.items())[:2], indent=4)

[   (   'top_A647_1',
        {   'location_center_x': 1571.0215596330274,
            'location_center_y': 1079.8862385321102,
            'plate': 'slide3',
            'site': 'M34',
            'well': 'A2'}),
    (   'top_A647_2',
        {   'location_center_x': 209.55241635687733,
            'location_center_y': 1256.9267657992566,
            'plate': 'slide3',
            'site': 'M26',
            'well': 'B2'})]


## Generate single-cell crops and save

In [11]:
generate_sc_crops(sc_dict=sc_dict, images_dir=images_dir, output_img_dir=output_img_dir, crop_size=crop_size)